In [13]:
import os, time, random, re, json, configparser
import pandas as pd
import numpy as np
from pandas import json_normalize

In [14]:
df_profile = pd.read_csv("../scraped_data/profile.csv", index_col=False)
df_experience = pd.read_csv("../scraped_data/experience.csv", index_col=False)
df_education = pd.read_csv("../scraped_data/education.csv", index_col=False)
df_certification = pd.read_csv("../scraped_data/certification.csv", index_col=False)
df_courses = pd.read_csv("../scraped_data/courses.csv", index_col=False)

In [15]:
df_profile = df_profile.replace("’", "'", regex=True).replace('&amp;','&', regex=True)
df_experience = df_experience.replace("’", "'", regex=True).replace('&amp;','&', regex=True)
df_education = df_education.replace("’", "'", regex=True).replace('&amp;','&', regex=True)
df_certification = df_certification.replace("’", "'", regex=True).replace('&amp;','&', regex=True)
df_courses = df_courses.replace("’", "'", regex=True).replace('&amp;','&', regex=True)

df_education.to_csv("experiment_education.csv", index=False)

### Profile 

In [16]:
# df_profile.head()

In [17]:
# df_profile["location"] = df_profile["location"].str.replace(", Bavaria", "")
df_profile["country"] = df_profile["location"].str.split(',').str[-1]
# df_profile

### Experience

In [18]:
jobtitle_map = {
    "https"                                                     : "Online Company",
    "Praktikant"                                                : "Student",
    "Research Assistant"                                        : "Student",
    "Teaching Assistant"                                        : "Student",
    "Wissenschaftliche Hilfskraft"                              : "Student",
    "Wissenschaftlicher Mitarbeiter"                            : "Researcher",
    "Wissenschaftliche Mitarbeiterin"                           : "Researcher",
    "Research Associate"                                        : "Researcher",
    "Postdoctoral Research Fellow"                              : "Researcher",
    "Postdoctoral Researcher"                                   : "Researcher",
    "Senior Consultant"                                         : "Consultant",
    "Research Assistant - Institute for Product Engineering"    : "Researcher",
    "Doctoral Researcher"                                       : "Doctoral Candidate",
}

r_jobtitle_map = {
    r"Master Thesis.*"                                          : "Student",
    r".*Student.*"                                              : "Student",
    r".*Intern.*"                                               : "Student",
    r"Senior Data.*Analyst.*"                                   : "Senior Data Analyst",
    r"Data.*[Ss]cien.*"                                         : "Data Scientist",
    r".*Software.*"                                             : "Software Engineer",
    r".*Data.*Engineer.*"                                       : "Data Engineer",
}

df_experience['job_title'].replace('', np.nan, inplace=True)
df_experience.dropna(subset=['job_title'], inplace=True)

df_experience["job_title"] = df_experience["job_title"].replace(jobtitle_map, regex=False)
# df_experience["job_title"] = df_experience["job_title"].replace(r_jobtitle_map, regex=True)

# df_experience["job_title"] = df_experience["job_title"].str.replace(r".*Data Analyst.*", "Data Analyst", regex=True)
# df_experience["job_title"] = df_experience["job_title"].str.replace(r".*Data Scientist.*", "Data Scientist", regex=True)
# df_experience["job_title"] = df_experience["job_title"].str.replace(r".*Data Engineer.*", "Data Engineer", regex=True)

### Educaiton

In [19]:
print(df_education.shape)

(1088, 4)


In [20]:
df_education["title"] = df_education["title"].str.title()
df_education["title"] = df_education["title"].str.replace("'S", "'s", regex=True).replace("Of", "of", regex=True)

df_education.to_csv("experiment_education.csv", index=False)
# M.Sc.

In [21]:
print(df_education.shape)

(1088, 4)


In [22]:
education_map = {
    "Doktor"                                : "Ph.D.",
    "Doctor of Philosophy"                  : "Ph.D.",
    "Doctoral Degree"                       : "Ph.D.",
    "Master of Science"                     : "Masters",
    "Master of Engineering"                 : "Masters",
    "Master's Degree"                       : "Masters",
    "Master of Arts"                        : "Masters",
    "Master"                                : "Masters",
    "Msc"                                   : "Masters",
    "M. Sc."                                : "Masters",
    "M.Sc."                                 : "Masters",
    "M.Sc"                                  : "Masters",
    "Bachelor of Science"                   : "Bachelors",
    "Bachelor's Degree"                     : "Bachelors",
    "Bachelor of Engineering"               : "Bachelors",
    "Bachelor of Arts"                      : "Bachelors",
    "Bachelor of Technology"                : "Bachelors",
    "Bachelor of Applied Science"           : "Bachelors",
    "Bachelor"                              : "Bachelors",
    "B.Sc."                                 : "Bachelors",
    "Bs"                                    : "Bachelors",
    "Dipl."                                 : "Bachelors",
    "Diplom"                                : "Bachelors",
    "Diploma"                               : "Bachelors",
    "Master of Business Administration"     : "MBA",
    "Bachelor of Business Administration"   : "BBA",
}

r_education_map = {
    r"Dr\. R.*"                             : "Ph.D.",
    r"Phd.*"                                : "Ph.D.",
    r"Master of.*"                          : "Masters",
}

def get_specialization(title):
    l = title.split(',') if not pd.isna(title) else ["", ""]
    stitle = l[0].split('-')[0].strip() if '-' in l[0] else l[0].strip()
    special = l[-1].strip()
    return [stitle, special]

df = pd.DataFrame(df_education["title"].apply(lambda title: get_specialization(title)).tolist(), columns=["degree", "specialization"])

df_education = pd.concat([df_education, df], axis=1).reindex(df_education.index)

df_education['degree'].replace('', np.nan, inplace=True)
df_education.dropna(subset=['degree'], inplace=True)

df_education["degree"] = df_education["degree"].replace(education_map, regex=False)
df_education["degree"] = df_education["degree"].replace(r_education_map, regex=True)

# df_education.to_csv("experiment_education.csv", index=False)
# df

In [23]:
print(df_education.shape)

(1058, 6)


### Certification

In [24]:
df_certification[['title', 'institute']].replace('', np.nan, inplace=True)
df_certification.dropna(subset=['title', 'institute'], inplace=True)

df_certification["institute"] = df_certification["institute"].str.replace(r".*Coursera.*", "Coursera", regex=True)

C:\Users\cn4tvne\AppData\Local\Temp\ipykernel_47872\1331510492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_certification[['title', 'institute']].replace('', np.nan, inplace=True)


### Save clean data

In [25]:
df_profile.to_csv("../clean_data/profile.csv", index=False)
df_experience.to_csv("../clean_data/experience.csv", index=False)
df_education.to_csv("../clean_data/education.csv", index=False)
df_certification.to_csv("../clean_data/certification.csv", index=False)
df_courses.to_csv("../clean_data/courses.csv", index=False)
print("done")

done
